In [1]:
pip install mediapipe

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install screen-brightness-control

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [3]:
pip install opencv-python mediapipe


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [4]:
import cv2
import mediapipe as mp

In [13]:
import screen_brightness_control as sbc


In [14]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Hands
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.75,
    min_tracking_confidence=0.75,
    max_num_hands=2)
Draw = mp.solutions.drawing_utils

# Capture video from webcam
cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    process = hands.process(frameRGB)
    
    landmarkList = []

    if process.multi_hand_landmarks:
        for handlm in process.multi_hand_landmarks:
            for _id, landmarks in enumerate(handlm.landmark):
                height, width, color_channels = frame.shape
                x, y = int(landmarks.x * width), int(landmarks.y * height)
                landmarkList.append([_id, x, y])
            Draw.draw_landmarks(frame, handlm, mpHands.HAND_CONNECTIONS)

    if landmarkList:
        x_1, y_1 = landmarkList[4][1], landmarkList[4][2]  # Thumb tip
        x_2, y_2 = landmarkList[8][1], landmarkList[8][2]  # Index finger tip
        
        cv2.circle(frame, (x_1, y_1), 7, (0, 255, 0), cv2.FILLED)
        cv2.circle(frame, (x_2, y_2), 7, (0, 255, 0), cv2.FILLED)  # Corrected to use x_2
        cv2.line(frame, (x_1, y_1), (x_2, y_2), (0, 255, 0), 3)

        # Calculate distance between thumb and index finger
        L = np.hypot(x_2 - x_1, y_2 - y_1)
        b_level = np.interp(L, [15, 220], [0, 100])
        # Assuming sbc.set_brightness is defined elsewhere
        sbc.set_brightness(int(b_level))

    cv2.imshow("Image", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
